In [ ]:
%%capture
import os
from pathlib import Path
import pandas as pd
from dj_notebook import activate

env_file = os.environ["META_ENV"]
reports_folder = Path(os.environ["META_REPORTS_FOLDER"])
analysis_folder = Path(os.environ["META_ANALYSIS_FOLDER"])
pharmacy_folder = Path(os.environ["META_PHARMACY_FOLDER"])
plus = activate(dotenv_file=env_file)

In [ ]:
from edc_pdutils.dataframes.get_subject_visit import convert_visit_code_to_float
from edc_pdutils.dataframes import get_crf
from edc_model_to_dataframe import read_frame_edc
from meta_subject.models import FollowupExamination


In [ ]:

df = read_frame_edc(FollowupExamination.objects.all(), drop_sys_columns=True, drop_action_item_columns=True)
df = df.replace("none", pd.NA)
df = df.replace("none", pd.NA)
df = df.fillna(pd.NA)
convert_visit_code_to_float(df)

In [ ]:
from edc_analytics.stata import get_stata_labels_from_model

df = df[["subject_identifier", "subject_visit_id", "report_datetime", "visit_code", "site_id", "site_name", "visit_reason", "symptoms","symptoms_detail", "symptoms_sought_care", "symptoms_g3", "symptoms_g4", "comment"]].copy().reset_index(drop=True)

df = df.astype(
    {col: "Float64" for col in df.select_dtypes(include=["float", "float64"]).columns}
)
df_meds = df.astype(
    {col: "Int64" for col in df.select_dtypes(include=["int", "int64"]).columns}
)
df = df.astype(
    {
        col: "datetime64[ns]"
        for col in df.select_dtypes(include=["datetime", "datetime64"]).columns
    }
)
df = df.astype(
    {
        col: str
        for col in df.select_dtypes(include=["object"]).columns
    }
)
df = df.fillna(pd.NA)

variable_labels = {}
variable_labels.update(**get_stata_labels_from_model(df, model="meta_subject.followupexamination", suffix=None))

df.to_stata(
    path=analysis_folder / "followupexamination.dta",
    variable_labels=variable_labels,
    version=118,
    write_index=False,
)

In [ ]:
# with get_crf
df2 = get_crf(model="meta_subject.followupexamination", subject_visit_model="meta_subject.subjectvisit")


In [ ]:
df2.to_stata(
    path=analysis_folder / "followupexamination.dta",
    variable_labels=variable_labels,
    version=118,
    write_index=False,
)